In [1]:
from datetime import datetime
from datetime import timedelta
import json
import re
from os import path, environ
from time import sleep
import os

import requests
#from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver      # imports
from selenium.webdriver.support.ui import Select

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5"
}
base_url = 'https://www.gunviolencearchive.org'

In [3]:
#These files need to be in the same directory as the jupyter notebook file.
SHOOTINGS_JSON = 'shootings.json'

SHOOTINGS_COMPACT_JSON = 'shootings_compact.json'

### Function declarations:

In [31]:

def scrape_results(current_url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5"
}
    url = current_url
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, features="lxml")
    table = soup.find("table")
    headers = [
        re.sub(r"[^a-z]", "", th.get_text().lower()) for th in table.find_all("th")
    ]
    rows = table.find("tbody").find_all("tr")
    #print(rows)
    results = []
    for row in rows:
        try:
            cols = row.find_all("td")
            #print(cols)
            links = row.select(".links a")
            incident_id = int(re.sub(r"[^\d]", "", links[0].get("href")))
            if not incident_id > 0:
                raise RuntimeError(f"Invalid incident ID {incident_id} at {url}")
            results.append(
                {
                    "id": int(re.sub(r"[^\d]", "", links[0].get("href"))),
                    "date": parse(cols[headers.index("incidentdate")].get_text()),
                    "state": cols[headers.index("state")].get_text(),
                    "city": cols[headers.index("cityorcounty")].get_text(),
                    "address": cols[headers.index("address")].get_text(),
                    "killed": int(cols[headers.index("killed")].get_text()),
                    "injured": int(cols[headers.index("injured")].get_text()),
                    "source": links[1].get("href") if len(links) > 1 else None,
                }
            )
        except:
            continue
        
    return results

In [27]:
def merge_shooting_data(scraped_data):
    total_new = 0
    for shooting in scraped_data:
        if shooting["id"] not in keyed_shooting_data:
            print(f'New shooting found: {shooting["id"]}, {shooting["date"]}')
            keyed_shooting_data[shooting["id"]] = {}
            total_new += 1
        keyed_shooting_data[shooting["id"]].update(shooting)
    return total_new

In [36]:

def parse_page(next_url):
    # HTTP GET requests
    page = requests.get(next_url)

    # Checking if we successfully fetched the URL
    if page.status_code == requests.codes.ok:
        bs = BeautifulSoup(page.text, 'lxml')
        try:
            current_url = next_url
            print(current_url)
            results = scrape_results(current_url)
            # Check for empty result:
            if not results:
                return "empty"
            else:
                #print(results)
                merge_shooting_data(results)
                next_page_text = bs.find('ul', class_="pager").findAll('li')[-2].text
                if next_page_text == 'next ›':
                    next_page_partial = bs.find('ul', class_="pager").findAll('li')[-2].find('a')['href']
                    next_page_url = base_url + next_page_partial
                    current_url = next_page_url
                    print(next_page_url)
                    results = scrape_results(current_url)
                    merge_shooting_data(results)
                    sleep(3)
                    if next_page_text == 'next ›':
                        parse_page(next_page_url)
                        sleep(1)
                    else:
                        merge_shooting_data(results)
        except:
            current_url = next_url
            results = scrape_results(current_url)
            merge_shooting_data(results)
            

### Main implementation:

In [37]:
# Test
year = '2019'
month= 'Jan'
day = 1


driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

driver.get('https://www.gunviolencearchive.org/query')



add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
add_rule_bar2.click()
sleep(4)


# Choose Date

driver.find_element_by_link_text('Date').click()

# 1 seconds for page loading
sleep(1)

# From DATE
# pull out calender
from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
from_date.click()
# month select
month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
sleep(1)
month1.select_by_visible_text(month)
sleep(1)


# year select
year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
sleep(1)
year1.select_by_visible_text(str(year))

sleep(1)


# start date

start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day]
start_date.click()

# TO DATE
# pull out calender
# end date
end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
end_date.click()

# month select
sleep(1)
month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
sleep(1)
month2.select_by_visible_text(month)
sleep(1)


# year select
year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
sleep(1)
year2.select_by_visible_text(str(year))

sleep(1)

# End date
# Currently, I made up first available date, which 0 index from the list.
End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day]
End_date.click()

sleep(1)

# Click add a rule
add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
add_rule_bar.click()
sleep(1)

# Select 'Incident Characteristic'

driver.find_element_by_link_text('Incident Characteristic').click()

# select Accidental Shooting
sleep(1)
acc_shoot = Select(driver.find_element_by_xpath("/html/body/section/div/div/div/div[2]/div/form/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/span/select"))
sleep(1)
acc_shoot.select_by_visible_text("Accidental Shooting")
# Click search
driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()

current_year = datetime.now().year
with open(SHOOTINGS_JSON) as json_file:
    shooting_data = json.load(json_file)
keyed_shooting_data = {}
for shooting in shooting_data:
    keyed_shooting_data[shooting["id"]] = shooting
    shooting["date"] = parse(shooting["date"])

print(f"Loading existing shooting data with {len(shooting_data)} incidents")
current_url = driver.current_url
parse_page(current_url)

Loading existing shooting data with 27 incidents
https://www.gunviolencearchive.org/query/4deebdc0-0869-4855-ba58-c74b065dcb50
empty


In [39]:
year = ['2019']
month = ['Jan']


for years in range(len(year)):
    for months in range(len(month)):
        
        if year[years]=='2020' or year[years]=='2016':
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        else:
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27] 
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        for days in range(len(day)):
            
            c_date = '{}-{}-{}'.format(year[years],month[months],day[days]+1)
            c_date = datetime.strptime(c_date,"%Y-%b-%d")
            day_difference = (datetime.now() - c_date).days
            if (day_difference >3):
                driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

                driver.get('https://www.gunviolencearchive.org/query')
                
                
                
                add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar2.click()
                sleep(4)
                
                
                # Choose Date
                
                driver.find_element_by_link_text('Date').click()
                
                # 1 seconds for page loading
                sleep(1)
                
                # From DATE
                # pull out calender
                from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
                from_date.click()
                # month select
                month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month1.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year1.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                
                # start date
                
                start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                start_date.click()
                
                # TO DATE
                # pull out calender
                # end date
                end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
                end_date.click()
                
                # month select
                sleep(1)
                month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month2.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year2.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                # End date
                # Currently, I made up first available date, which 0 index from the list.
                End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                End_date.click()
                
                sleep(1)
                
                # Click add a rule
                add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar.click()
                sleep(1)
                
                # Select 'Incident Characteristic'
                
                driver.find_element_by_link_text('Incident Characteristic').click()
                # select Accidental Shooting
                sleep(1)
                acc_shoot = Select(driver.find_element_by_xpath("/html/body/section/div/div/div/div[2]/div/form/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/span/select"))
                sleep(1)
                acc_shoot.select_by_visible_text("Accidental Shooting")
                
                # Click search
                driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()
                
                start_year = 2014
                current_year = datetime.now().year
                with open(SHOOTINGS_JSON) as json_file:
                    shooting_data = json.load(json_file)

                keyed_shooting_data = {}
                for shooting in shooting_data:
                    keyed_shooting_data[shooting["id"]] = shooting
                    shooting["date"] = parse(shooting["date"])

                print(f"Loading existing shooting data with {len(shooting_data)} incidents")
                current_url = driver.current_url
                if parse_page(current_url)=='empty':
                    continue
                else:
                    parse_page(current_url)
                    print("Sorting results")
                    updated_shooting_data = sorted(
                        keyed_shooting_data.values(), key=lambda row: row['date']
                    )
    
                    updated_shootings_compact = []
                    for shooting in updated_shooting_data:
                        shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
                        updated_shootings_compact.append(
                            [
                                shooting["id"],
                                shooting["date"],
                                shooting["killed"],
                                shooting["injured"],
    
                            ]
                        )
    
    
                    print("Writing results")
                    with open(SHOOTINGS_JSON, "w") as outfile:
                        json.dump(updated_shooting_data, outfile, indent=2)
    
                    with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
                        json.dump(updated_shootings_compact, outfile, separators=(",", ":"))
    
                    print(f"Done! {len(updated_shooting_data)} shootings found")
                    driver.close()
            else:
                break
            


Loading existing shooting data with 27 incidents
https://www.gunviolencearchive.org/query/534403d0-a48b-4898-b725-19b455f0add0
https://www.gunviolencearchive.org/query/534403d0-a48b-4898-b725-19b455f0add0?page=1
https://www.gunviolencearchive.org/query/534403d0-a48b-4898-b725-19b455f0add0?page=1
https://www.gunviolencearchive.org/query/534403d0-a48b-4898-b725-19b455f0add0
https://www.gunviolencearchive.org/query/534403d0-a48b-4898-b725-19b455f0add0?page=1
https://www.gunviolencearchive.org/query/534403d0-a48b-4898-b725-19b455f0add0?page=1
Sorting results
Writing results
Done! 27 shootings found
Loading existing shooting data with 27 incidents
https://www.gunviolencearchive.org/query/8f776653-7e11-4521-9a57-61d5ccb632d8
empty
https://www.gunviolencearchive.org/query/8f776653-7e11-4521-9a57-61d5ccb632d8
empty
Sorting results
Writing results
Done! 27 shootings found
Loading existing shooting data with 27 incidents
https://www.gunviolencearchive.org/query/c9123b94-9fd0-4da2-ad94-f4b7b51b60

https://www.gunviolencearchive.org/query/4fe547ac-d3c8-4d59-8c53-5a9e18405aa1
New shooting found: 1304942, 2019-01-18 00:00:00
New shooting found: 1304577, 2019-01-18 00:00:00
New shooting found: 1304662, 2019-01-18 00:00:00
New shooting found: 1304982, 2019-01-18 00:00:00
https://www.gunviolencearchive.org/query/4fe547ac-d3c8-4d59-8c53-5a9e18405aa1
Sorting results
Writing results
Done! 100 shootings found
Loading existing shooting data with 100 incidents
https://www.gunviolencearchive.org/query/d32a04e3-bdf7-4d39-b6c5-2938c1b0e692
New shooting found: 1306720, 2019-01-19 00:00:00
New shooting found: 1306011, 2019-01-19 00:00:00
New shooting found: 1308718, 2019-01-19 00:00:00
New shooting found: 1306395, 2019-01-19 00:00:00
New shooting found: 1306941, 2019-01-19 00:00:00
New shooting found: 1309253, 2019-01-19 00:00:00
New shooting found: 1498906, 2019-01-19 00:00:00
New shooting found: 1386508, 2019-01-19 00:00:00
New shooting found: 1305620, 2019-01-19 00:00:00
New shooting found: 1

In [40]:
year = ['2019']
month = ['Feb']


for years in range(len(year)):
    for months in range(len(month)):
        
        if year[years]=='2020' or year[years]=='2016':
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        else:
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27] 
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        for days in range(len(day)):
            
            c_date = '{}-{}-{}'.format(year[years],month[months],day[days]+1)
            c_date = datetime.strptime(c_date,"%Y-%b-%d")
            day_difference = (datetime.now() - c_date).days
            if (day_difference >3):
                driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

                driver.get('https://www.gunviolencearchive.org/query')
                
                
                
                add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar2.click()
                sleep(4)
                
                
                # Choose Date
                
                driver.find_element_by_link_text('Date').click()
                
                # 1 seconds for page loading
                sleep(1)
                
                # From DATE
                # pull out calender
                from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
                from_date.click()
                # month select
                month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month1.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year1.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                
                # start date
                
                start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                start_date.click()
                
                # TO DATE
                # pull out calender
                # end date
                end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
                end_date.click()
                
                # month select
                sleep(1)
                month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month2.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year2.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                # End date
                # Currently, I made up first available date, which 0 index from the list.
                End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                End_date.click()
                
                sleep(1)
                
                # Click add a rule
                add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar.click()
                sleep(1)
                
                # Select 'Incident Characteristic'
                
                driver.find_element_by_link_text('Incident Characteristic').click()
                # select Accidental Shooting
                sleep(1)
                acc_shoot = Select(driver.find_element_by_xpath("/html/body/section/div/div/div/div[2]/div/form/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/span/select"))
                sleep(1)
                acc_shoot.select_by_visible_text("Accidental Shooting")
                
                # Click search
                driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()
                
                start_year = 2014
                current_year = datetime.now().year
                with open(SHOOTINGS_JSON) as json_file:
                    shooting_data = json.load(json_file)

                keyed_shooting_data = {}
                for shooting in shooting_data:
                    keyed_shooting_data[shooting["id"]] = shooting
                    shooting["date"] = parse(shooting["date"])

                print(f"Loading existing shooting data with {len(shooting_data)} incidents")
                current_url = driver.current_url
                if parse_page(current_url)=='empty':
                    continue
                else:
                    parse_page(current_url)
                    print("Sorting results")
                    updated_shooting_data = sorted(
                        keyed_shooting_data.values(), key=lambda row: row['date']
                    )
    
                    updated_shootings_compact = []
                    for shooting in updated_shooting_data:
                        shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
                        updated_shootings_compact.append(
                            [
                                shooting["id"],
                                shooting["date"],
                                shooting["killed"],
                                shooting["injured"],
    
                            ]
                        )
    
    
                    print("Writing results")
                    with open(SHOOTINGS_JSON, "w") as outfile:
                        json.dump(updated_shooting_data, outfile, indent=2)
    
                    with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
                        json.dump(updated_shootings_compact, outfile, separators=(",", ":"))
    
                    print(f"Done! {len(updated_shooting_data)} shootings found")
                    driver.close()
            else:
                break
            


Loading existing shooting data with 161 incidents
https://www.gunviolencearchive.org/query/e38e85cd-43f1-449a-8ddc-c68c5423ff73
New shooting found: 1322220, 2019-02-01 00:00:00
https://www.gunviolencearchive.org/query/e38e85cd-43f1-449a-8ddc-c68c5423ff73
Sorting results
Writing results
Done! 162 shootings found
Loading existing shooting data with 162 incidents
https://www.gunviolencearchive.org/query/cc34fdbe-9f03-4c3d-9b98-23cf4f4c966f
New shooting found: 1318638, 2019-02-02 00:00:00
New shooting found: 1318833, 2019-02-02 00:00:00
New shooting found: 1318875, 2019-02-02 00:00:00
https://www.gunviolencearchive.org/query/cc34fdbe-9f03-4c3d-9b98-23cf4f4c966f
Sorting results
Writing results
Done! 165 shootings found
Loading existing shooting data with 165 incidents
https://www.gunviolencearchive.org/query/9ebebca4-6eda-4052-8c94-54397bc4deca
New shooting found: 1318461, 2019-02-03 00:00:00
New shooting found: 1319452, 2019-02-03 00:00:00
New shooting found: 1319137, 2019-02-03 00:00:00
N

New shooting found: 1331029, 2019-02-19 00:00:00
New shooting found: 1330716, 2019-02-19 00:00:00
New shooting found: 1331529, 2019-02-19 00:00:00
New shooting found: 1330843, 2019-02-19 00:00:00
https://www.gunviolencearchive.org/query/4d29a15f-1d7f-44d2-b1d6-88577cdfa476
Sorting results
Writing results
Done! 234 shootings found
Loading existing shooting data with 234 incidents
https://www.gunviolencearchive.org/query/255bb1dc-5f0b-4c48-8a69-eae4b48ea40b
New shooting found: 1331403, 2019-02-20 00:00:00
New shooting found: 1331052, 2019-02-20 00:00:00
New shooting found: 1331207, 2019-02-20 00:00:00
https://www.gunviolencearchive.org/query/255bb1dc-5f0b-4c48-8a69-eae4b48ea40b
Sorting results
Writing results
Done! 237 shootings found
Loading existing shooting data with 237 incidents
https://www.gunviolencearchive.org/query/ed3d4065-09f5-40ad-b655-23cedcc01875
New shooting found: 1332265, 2019-02-21 00:00:00
New shooting found: 1332270, 2019-02-21 00:00:00
New shooting found: 1332190, 20

In [47]:
# 'Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'

year = ['2019']
month = ['Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']


for years in range(len(year)):
    for months in range(len(month)):
        
        if year[years]=='2020' or year[years]=='2016':
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        else:
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27] 
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        for days in range(len(day)):
            
            c_date = '{}-{}-{}'.format(year[years],month[months],day[days]+1)
            c_date = datetime.strptime(c_date,"%Y-%b-%d")
            day_difference = (datetime.now() - c_date).days
            if (day_difference >3):
                driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

                driver.get('https://www.gunviolencearchive.org/query')
                
                
                
                add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar2.click()
                sleep(4)
                
                
                # Choose Date
                
                driver.find_element_by_link_text('Date').click()
                
                # 1 seconds for page loading
                sleep(1)
                
                # From DATE
                # pull out calender
                from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
                from_date.click()
                # month select
                month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month1.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year1.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                
                # start date
                
                start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                start_date.click()
                
                # TO DATE
                # pull out calender
                # end date
                end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
                end_date.click()
                
                # month select
                sleep(1)
                month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month2.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year2.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                # End date
                # Currently, I made up first available date, which 0 index from the list.
                End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                End_date.click()
                
                sleep(1)
                
                # Click add a rule
                add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar.click()
                sleep(1)
                
                # Select 'Incident Characteristic'
                
                driver.find_element_by_link_text('Incident Characteristic').click()
                # select Accidental Shooting
                sleep(1)
                acc_shoot = Select(driver.find_element_by_xpath("/html/body/section/div/div/div/div[2]/div/form/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/span/select"))
                sleep(1)
                acc_shoot.select_by_visible_text("Accidental Shooting")
                
                # Click search
                driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()
                
                start_year = 2014
                current_year = datetime.now().year
                with open(SHOOTINGS_JSON) as json_file:
                    shooting_data = json.load(json_file)

                keyed_shooting_data = {}
                for shooting in shooting_data:
                    keyed_shooting_data[shooting["id"]] = shooting
                    shooting["date"] = parse(shooting["date"])

                print(f"Loading existing shooting data with {len(shooting_data)} incidents")
                current_url = driver.current_url
                if parse_page(current_url)=='empty':
                    continue
                else:
                    parse_page(current_url)
                    print("Sorting results")
                    updated_shooting_data = sorted(
                        keyed_shooting_data.values(), key=lambda row: row['date']
                    )
    
                    updated_shootings_compact = []
                    for shooting in updated_shooting_data:
                        shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
                        updated_shootings_compact.append(
                            [
                                shooting["id"],
                                shooting["date"],
                                shooting["killed"],
                                shooting["injured"],
    
                            ]
                        )
    
    
                    print("Writing results")
                    with open(SHOOTINGS_JSON, "w") as outfile:
                        json.dump(updated_shooting_data, outfile, indent=2)
    
                    with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
                        json.dump(updated_shootings_compact, outfile, separators=(",", ":"))
    
                    print(f"Done! {len(updated_shooting_data)} shootings found")
                    driver.close()
            else:
                break
            


Loading existing shooting data with 1519 incidents
https://www.gunviolencearchive.org/query/3283f519-b47c-42fb-9bdf-3332cd99964c
New shooting found: 1539793, 2019-11-01 00:00:00
New shooting found: 1539930, 2019-11-01 00:00:00
New shooting found: 1539761, 2019-11-01 00:00:00
New shooting found: 1539858, 2019-11-01 00:00:00
New shooting found: 1540165, 2019-11-01 00:00:00
New shooting found: 1541353, 2019-11-01 00:00:00
New shooting found: 1552483, 2019-11-01 00:00:00
New shooting found: 1540100, 2019-11-01 00:00:00
New shooting found: 1592715, 2019-11-01 00:00:00
https://www.gunviolencearchive.org/query/3283f519-b47c-42fb-9bdf-3332cd99964c
Sorting results
Writing results
Done! 1528 shootings found
Loading existing shooting data with 1528 incidents
https://www.gunviolencearchive.org/query/b016294c-099a-4f1e-a799-a40f6c1ca3a5
New shooting found: 1541084, 2019-11-02 00:00:00
New shooting found: 1541309, 2019-11-02 00:00:00
New shooting found: 1541956, 2019-11-02 00:00:00
New shooting foun

New shooting found: 1551449, 2019-11-16 00:00:00
New shooting found: 1551302, 2019-11-16 00:00:00
New shooting found: 1552050, 2019-11-16 00:00:00
New shooting found: 1551193, 2019-11-16 00:00:00
New shooting found: 1551098, 2019-11-16 00:00:00
New shooting found: 1551879, 2019-11-16 00:00:00
New shooting found: 1550420, 2019-11-16 00:00:00
New shooting found: 1550826, 2019-11-16 00:00:00
https://www.gunviolencearchive.org/query/5df0190c-587f-4b7c-a114-8c34958fc1bf
Sorting results
Writing results
Done! 1611 shootings found
Loading existing shooting data with 1611 incidents
https://www.gunviolencearchive.org/query/42b8dd12-87a0-4a45-badb-0fbe72974527
New shooting found: 1553247, 2019-11-17 00:00:00
New shooting found: 1551972, 2019-11-17 00:00:00
New shooting found: 1552815, 2019-11-17 00:00:00
New shooting found: 1551411, 2019-11-17 00:00:00
New shooting found: 1552021, 2019-11-17 00:00:00
New shooting found: 1551921, 2019-11-17 00:00:00
New shooting found: 1550986, 2019-11-17 00:00:00

https://www.gunviolencearchive.org/query/5a06418c-fa5c-4356-9b76-c9b210d4a69b
Sorting results
Writing results
Done! 1710 shootings found
Loading existing shooting data with 1710 incidents
https://www.gunviolencearchive.org/query/7390ce39-2385-49b1-9e51-2d5f30eda8a8
New shooting found: 1561428, 2019-11-29 00:00:00
New shooting found: 1561690, 2019-11-29 00:00:00
New shooting found: 1561284, 2019-11-29 00:00:00
https://www.gunviolencearchive.org/query/7390ce39-2385-49b1-9e51-2d5f30eda8a8
Sorting results
Writing results
Done! 1713 shootings found
Loading existing shooting data with 1713 incidents
https://www.gunviolencearchive.org/query/8cd7f4ca-2845-41ac-9692-f841b62b387a
New shooting found: 1561232, 2019-11-30 00:00:00
New shooting found: 1564261, 2019-11-30 00:00:00
New shooting found: 1562710, 2019-11-30 00:00:00
New shooting found: 1561893, 2019-11-30 00:00:00
https://www.gunviolencearchive.org/query/8cd7f4ca-2845-41ac-9692-f841b62b387a
Sorting results
Writing results
Done! 1717 shoo

https://www.gunviolencearchive.org/query/4e5b6597-58d4-4134-bbc1-fef8c51a0330
Sorting results
Writing results
Done! 1800 shootings found
Loading existing shooting data with 1800 incidents
https://www.gunviolencearchive.org/query/c8956b4a-7ccb-4ff7-b8de-1be5ae5eecbe
New shooting found: 1572059, 2019-12-14 00:00:00
New shooting found: 1574160, 2019-12-14 00:00:00
New shooting found: 1571814, 2019-12-14 00:00:00
New shooting found: 1572728, 2019-12-14 00:00:00
New shooting found: 1572606, 2019-12-14 00:00:00
New shooting found: 1572029, 2019-12-14 00:00:00
New shooting found: 1572632, 2019-12-14 00:00:00
https://www.gunviolencearchive.org/query/c8956b4a-7ccb-4ff7-b8de-1be5ae5eecbe
Sorting results
Writing results
Done! 1807 shootings found
Loading existing shooting data with 1807 incidents
https://www.gunviolencearchive.org/query/4ff6662b-afcf-4748-b799-2089151ecdf4
New shooting found: 1572827, 2019-12-15 00:00:00
New shooting found: 1572962, 2019-12-15 00:00:00
New shooting found: 1572180

https://www.gunviolencearchive.org/query/440a7cc7-d49e-45d1-addf-0c2434d459da
Sorting results
Writing results
Done! 1882 shootings found
Loading existing shooting data with 1882 incidents
https://www.gunviolencearchive.org/query/1d9c6a87-2070-4dfc-9421-75af9085d384
New shooting found: 1584355, 2019-12-30 00:00:00
https://www.gunviolencearchive.org/query/1d9c6a87-2070-4dfc-9421-75af9085d384
Sorting results
Writing results
Done! 1883 shootings found
Loading existing shooting data with 1883 incidents
https://www.gunviolencearchive.org/query/cb74707a-03de-4b7d-a6f4-534130ebe8fa
New shooting found: 1586912, 2019-12-31 00:00:00
New shooting found: 1585976, 2019-12-31 00:00:00
New shooting found: 1584998, 2019-12-31 00:00:00
New shooting found: 1585345, 2019-12-31 00:00:00
New shooting found: 1585159, 2019-12-31 00:00:00
New shooting found: 1586646, 2019-12-31 00:00:00
New shooting found: 1585456, 2019-12-31 00:00:00
https://www.gunviolencearchive.org/query/cb74707a-03de-4b7d-a6f4-534130ebe8f

In [48]:
#2020
year = ['2020']
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug']


for years in range(len(year)):
    for months in range(len(month)):
        
        if year[years]=='2020' or year[years]=='2016':
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        else:
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27] 
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        for days in range(len(day)):
            
            c_date = '{}-{}-{}'.format(year[years],month[months],day[days]+1)
            c_date = datetime.strptime(c_date,"%Y-%b-%d")
            day_difference = (datetime.now() - c_date).days
            if (day_difference >3):
                driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

                driver.get('https://www.gunviolencearchive.org/query')
                
                
                
                add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar2.click()
                sleep(4)
                
                
                # Choose Date
                
                driver.find_element_by_link_text('Date').click()
                
                # 1 seconds for page loading
                sleep(1)
                
                # From DATE
                # pull out calender
                from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
                from_date.click()
                # month select
                month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month1.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year1.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                
                # start date
                
                start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                start_date.click()
                
                # TO DATE
                # pull out calender
                # end date
                end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
                end_date.click()
                
                # month select
                sleep(1)
                month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month2.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year2.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                # End date
                # Currently, I made up first available date, which 0 index from the list.
                End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                End_date.click()
                
                sleep(1)
                
                # Click add a rule
                add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar.click()
                sleep(1)
                
                # Select 'Incident Characteristic'
                
                driver.find_element_by_link_text('Incident Characteristic').click()
                # select Accidental Shooting
                sleep(1)
                acc_shoot = Select(driver.find_element_by_xpath("/html/body/section/div/div/div/div[2]/div/form/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/span/select"))
                sleep(1)
                acc_shoot.select_by_visible_text("Accidental Shooting")
                
                # Click search
                driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()
                
                start_year = 2014
                current_year = datetime.now().year
                with open(SHOOTINGS_JSON) as json_file:
                    shooting_data = json.load(json_file)

                keyed_shooting_data = {}
                for shooting in shooting_data:
                    keyed_shooting_data[shooting["id"]] = shooting
                    shooting["date"] = parse(shooting["date"])

                print(f"Loading existing shooting data with {len(shooting_data)} incidents")
                current_url = driver.current_url
                if parse_page(current_url)=='empty':
                    continue
                else:
                    parse_page(current_url)
                    print("Sorting results")
                    updated_shooting_data = sorted(
                        keyed_shooting_data.values(), key=lambda row: row['date']
                    )
    
                    updated_shootings_compact = []
                    for shooting in updated_shooting_data:
                        shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
                        updated_shootings_compact.append(
                            [
                                shooting["id"],
                                shooting["date"],
                                shooting["killed"],
                                shooting["injured"],
    
                            ]
                        )
    
    
                    print("Writing results")
                    with open(SHOOTINGS_JSON, "w") as outfile:
                        json.dump(updated_shooting_data, outfile, indent=2)
    
                    with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
                        json.dump(updated_shootings_compact, outfile, separators=(",", ":"))
    
                    print(f"Done! {len(updated_shooting_data)} shootings found")
                    driver.close()
            else:
                break
            


Loading existing shooting data with 1890 incidents
https://www.gunviolencearchive.org/query/33dab098-cd9f-4cf1-9ecb-4dc021ec5b19
New shooting found: 1585484, 2020-01-01 00:00:00
New shooting found: 1586031, 2020-01-01 00:00:00
New shooting found: 1586119, 2020-01-01 00:00:00
New shooting found: 1585823, 2020-01-01 00:00:00
New shooting found: 1586872, 2020-01-01 00:00:00
New shooting found: 1598354, 2020-01-01 00:00:00
New shooting found: 1585741, 2020-01-01 00:00:00
New shooting found: 1586843, 2020-01-01 00:00:00
New shooting found: 1586238, 2020-01-01 00:00:00
New shooting found: 1585454, 2020-01-01 00:00:00
New shooting found: 1586682, 2020-01-01 00:00:00
New shooting found: 1587822, 2020-01-01 00:00:00
New shooting found: 1585503, 2020-01-01 00:00:00
New shooting found: 1587594, 2020-01-01 00:00:00
New shooting found: 1585419, 2020-01-01 00:00:00
New shooting found: 1585180, 2020-01-01 00:00:00
New shooting found: 1585315, 2020-01-01 00:00:00
New shooting found: 1585205, 2020-01-0

https://www.gunviolencearchive.org/query/316d4925-89da-4e26-a616-72fdf023ed10
Sorting results
Writing results
Done! 1978 shootings found
Loading existing shooting data with 1978 incidents
https://www.gunviolencearchive.org/query/491101c9-1fe7-4350-a79a-0ac6d291b330
New shooting found: 1598580, 2020-01-15 00:00:00
New shooting found: 1598379, 2020-01-15 00:00:00
New shooting found: 1597282, 2020-01-15 00:00:00
New shooting found: 1597760, 2020-01-15 00:00:00
New shooting found: 1597714, 2020-01-15 00:00:00
New shooting found: 1597962, 2020-01-15 00:00:00
New shooting found: 1597335, 2020-01-15 00:00:00
New shooting found: 1601933, 2020-01-15 00:00:00
New shooting found: 1597618, 2020-01-15 00:00:00
https://www.gunviolencearchive.org/query/491101c9-1fe7-4350-a79a-0ac6d291b330
Sorting results
Writing results
Done! 1987 shootings found
Loading existing shooting data with 1987 incidents
https://www.gunviolencearchive.org/query/6bdcdd6d-bb2c-4191-8464-3c777f0caf65
New shooting found: 1599211

New shooting found: 1608539, 2020-01-29 00:00:00
New shooting found: 1609285, 2020-01-29 00:00:00
New shooting found: 1608602, 2020-01-29 00:00:00
https://www.gunviolencearchive.org/query/4f7c2f64-2a2e-4240-95dd-aa1a4c765809
Sorting results
Writing results
Done! 2068 shootings found
Loading existing shooting data with 2068 incidents
https://www.gunviolencearchive.org/query/61089323-1344-46bb-acf5-b3c13f8b0524
New shooting found: 1609853, 2020-01-30 00:00:00
New shooting found: 1609445, 2020-01-30 00:00:00
New shooting found: 1609987, 2020-01-30 00:00:00
New shooting found: 1609513, 2020-01-30 00:00:00
New shooting found: 1610061, 2020-01-30 00:00:00
New shooting found: 1609952, 2020-01-30 00:00:00
https://www.gunviolencearchive.org/query/61089323-1344-46bb-acf5-b3c13f8b0524
Sorting results
Writing results
Done! 2074 shootings found
Loading existing shooting data with 2074 incidents
https://www.gunviolencearchive.org/query/bb492700-88ec-404d-aebc-8d8737683ff0
New shooting found: 1610345

Sorting results
Writing results
Done! 2150 shootings found
Loading existing shooting data with 2150 incidents
https://www.gunviolencearchive.org/query/f86d6407-ecb6-4731-ba30-c76ef8a343bb
New shooting found: 1621636, 2020-02-14 00:00:00
New shooting found: 1621936, 2020-02-14 00:00:00
New shooting found: 1621865, 2020-02-14 00:00:00
New shooting found: 1621834, 2020-02-14 00:00:00
New shooting found: 1621830, 2020-02-14 00:00:00
New shooting found: 1624362, 2020-02-14 00:00:00
https://www.gunviolencearchive.org/query/f86d6407-ecb6-4731-ba30-c76ef8a343bb
Sorting results
Writing results
Done! 2156 shootings found
Loading existing shooting data with 2156 incidents
https://www.gunviolencearchive.org/query/8316573f-b860-4a57-8394-1263e0513090
New shooting found: 1675445, 2020-02-15 00:00:00
New shooting found: 1622789, 2020-02-15 00:00:00
New shooting found: 1622312, 2020-02-15 00:00:00
New shooting found: 1623104, 2020-02-15 00:00:00
New shooting found: 1622267, 2020-02-15 00:00:00
New sho

https://www.gunviolencearchive.org/query/1a184d4f-72bd-4873-a546-22797f5b6931
Sorting results
Writing results
Done! 2235 shootings found
Loading existing shooting data with 2235 incidents
https://www.gunviolencearchive.org/query/f477d596-5511-4ad3-8619-39a700c33c84
New shooting found: 1632282, 2020-03-01 00:00:00
New shooting found: 1632401, 2020-03-01 00:00:00
New shooting found: 1631792, 2020-03-01 00:00:00
New shooting found: 1631780, 2020-03-01 00:00:00
New shooting found: 1632354, 2020-03-01 00:00:00
New shooting found: 1631705, 2020-03-01 00:00:00
New shooting found: 1632058, 2020-03-01 00:00:00
New shooting found: 1632100, 2020-03-01 00:00:00
New shooting found: 1662940, 2020-03-01 00:00:00
https://www.gunviolencearchive.org/query/f477d596-5511-4ad3-8619-39a700c33c84
Sorting results
Writing results
Done! 2244 shootings found
Loading existing shooting data with 2244 incidents
https://www.gunviolencearchive.org/query/0d558af9-0415-4d0b-a5f8-ccfa34376ad1
New shooting found: 1632677

...

https://www.gunviolencearchive.org/query/74471b73-d2d5-4300-9160-4830cf9c747b
New shooting found: 1710229, 2020-06-18 00:00:00
New shooting found: 1709255, 2020-06-18 00:00:00
New shooting found: 1709520, 2020-06-18 00:00:00
New shooting found: 1709730, 2020-06-18 00:00:00
New shooting found: 1711149, 2020-06-18 00:00:00
New shooting found: 1708789, 2020-06-18 00:00:00
https://www.gunviolencearchive.org/query/74471b73-d2d5-4300-9160-4830cf9c747b
Sorting results
Writing results
Done! 2826 shootings found
Loading existing shooting data with 2826 incidents
https://www.gunviolencearchive.org/query/04a1f28b-4a2f-40ec-ba9f-95501323600f
New shooting found: 1710124, 2020-06-19 00:00:00
New shooting found: 1714604, 2020-06-19 00:00:00
New shooting found: 1710081, 2020-06-19 00:00:00
New shooting found: 1710749, 2020-06-19 00:00:00
New shooting found: 1716156, 2020-06-19 00:00:00
New shooting found: 1716070, 2020-06-19 00:00:00
https://www.gunviolencearchive.org/query/04a1f28b-4a2f-40ec-ba9f-955

https://www.gunviolencearchive.org/query/eb2fae60-1e20-4912-a921-7439eecaadcc
Sorting results
Writing results
Done! 2911 shootings found
Loading existing shooting data with 2911 incidents
https://www.gunviolencearchive.org/query/fb572165-8aa4-4cf9-9b57-de08eb29ba88
New shooting found: 1724524, 2020-07-03 00:00:00
New shooting found: 1724110, 2020-07-03 00:00:00
New shooting found: 1723897, 2020-07-03 00:00:00
https://www.gunviolencearchive.org/query/fb572165-8aa4-4cf9-9b57-de08eb29ba88
Sorting results
Writing results
Done! 2914 shootings found
Loading existing shooting data with 2914 incidents
https://www.gunviolencearchive.org/query/d1e1c7f8-cb53-4002-8c35-c491a6ab3cc4
New shooting found: 1725094, 2020-07-04 00:00:00
New shooting found: 1724885, 2020-07-04 00:00:00
New shooting found: 1728179, 2020-07-04 00:00:00
New shooting found: 1725791, 2020-07-04 00:00:00
New shooting found: 1728182, 2020-07-04 00:00:00
New shooting found: 1728111, 2020-07-04 00:00:00
New shooting found: 1724737

https://www.gunviolencearchive.org/query/fc588b25-6c27-4595-aab2-a989cec436df
Sorting results
Writing results
Done! 3011 shootings found
Loading existing shooting data with 3011 incidents
https://www.gunviolencearchive.org/query/9acbc829-a402-48c7-a572-15650eb1d9a1
New shooting found: 1737437, 2020-07-16 00:00:00
New shooting found: 1737066, 2020-07-16 00:00:00
New shooting found: 1737281, 2020-07-16 00:00:00
New shooting found: 1752409, 2020-07-16 00:00:00
New shooting found: 1737194, 2020-07-16 00:00:00
New shooting found: 1737649, 2020-07-16 00:00:00
New shooting found: 1737557, 2020-07-16 00:00:00
New shooting found: 1737445, 2020-07-16 00:00:00
New shooting found: 1737502, 2020-07-16 00:00:00
New shooting found: 1737404, 2020-07-16 00:00:00
https://www.gunviolencearchive.org/query/9acbc829-a402-48c7-a572-15650eb1d9a1
Sorting results
Writing results
Done! 3021 shootings found
Loading existing shooting data with 3021 incidents
https://www.gunviolencearchive.org/query/0d58ee78-93a3-4

https://www.gunviolencearchive.org/query/44c3f5f6-eaf3-4e30-a064-de826c26a6a0
Sorting results
Writing results
Done! 3109 shootings found
Loading existing shooting data with 3109 incidents
https://www.gunviolencearchive.org/query/5f79393e-472a-4694-939f-ea70d9d7c8f1
New shooting found: 1750602, 2020-07-29 00:00:00
New shooting found: 1750109, 2020-07-29 00:00:00
New shooting found: 1749673, 2020-07-29 00:00:00
https://www.gunviolencearchive.org/query/5f79393e-472a-4694-939f-ea70d9d7c8f1
Sorting results
Writing results
Done! 3112 shootings found
Loading existing shooting data with 3112 incidents
https://www.gunviolencearchive.org/query/4879a72c-ee40-4671-877b-fea1c6433153
New shooting found: 1751841, 2020-07-30 00:00:00
New shooting found: 1751618, 2020-07-30 00:00:00
New shooting found: 1751102, 2020-07-30 00:00:00
New shooting found: 1751070, 2020-07-30 00:00:00
https://www.gunviolencearchive.org/query/4879a72c-ee40-4671-877b-fea1c6433153
Sorting results
Writing results
Done! 3116 shoo

https://www.gunviolencearchive.org/query/5e9e352b-e4a6-42a1-83f4-233ab8c705b5
Sorting results
Writing results
Done! 3205 shootings found
Loading existing shooting data with 3205 incidents
https://www.gunviolencearchive.org/query/d998167b-05dc-4da3-b587-4029cfc55612
New shooting found: 1762479, 2020-08-12 00:00:00
New shooting found: 1762893, 2020-08-12 00:00:00
New shooting found: 1763278, 2020-08-12 00:00:00
New shooting found: 1762990, 2020-08-12 00:00:00
New shooting found: 1763418, 2020-08-12 00:00:00
New shooting found: 1763392, 2020-08-12 00:00:00
New shooting found: 1763384, 2020-08-12 00:00:00
New shooting found: 1764086, 2020-08-12 00:00:00
New shooting found: 1763889, 2020-08-12 00:00:00
https://www.gunviolencearchive.org/query/d998167b-05dc-4da3-b587-4029cfc55612
Sorting results
Writing results
Done! 3214 shootings found
Loading existing shooting data with 3214 incidents
https://www.gunviolencearchive.org/query/b48c896a-182d-4e8a-872e-b476a37d4d29
New shooting found: 1763742

In [52]:
#2018

#'Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',
year = ['2018']
month = ['Sep','Oct','Nov','Dec']


for years in range(len(year)):
    for months in range(len(month)):
        
        if year[years]=='2020' or year[years]=='2016':
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        else:
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27] 
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        for days in range(len(day)):
            
            c_date = '{}-{}-{}'.format(year[years],month[months],day[days]+1)
            c_date = datetime.strptime(c_date,"%Y-%b-%d")
            day_difference = (datetime.now() - c_date).days
            if (day_difference >3):
                driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

                driver.get('https://www.gunviolencearchive.org/query')
                
                
                
                add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar2.click()
                sleep(4)
                
                
                # Choose Date
                
                driver.find_element_by_link_text('Date').click()
                
                # 1 seconds for page loading
                sleep(1)
                
                # From DATE
                # pull out calender
                from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
                from_date.click()
                # month select
                month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month1.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year1.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                
                # start date
                
                start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                start_date.click()
                
                # TO DATE
                # pull out calender
                # end date
                end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
                end_date.click()
                
                # month select
                sleep(1)
                month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month2.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year2.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                # End date
                # Currently, I made up first available date, which 0 index from the list.
                End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                End_date.click()
                
                sleep(1)
                
                # Click add a rule
                add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar.click()
                sleep(1)
                
                # Select 'Incident Characteristic'
                
                driver.find_element_by_link_text('Incident Characteristic').click()
                # select Accidental Shooting
                sleep(1)
                acc_shoot = Select(driver.find_element_by_xpath("/html/body/section/div/div/div/div[2]/div/form/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/span/select"))
                sleep(1)
                acc_shoot.select_by_visible_text("Accidental Shooting")
                
                # Click search
                driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()
                
                start_year = 2014
                current_year = datetime.now().year
                with open(SHOOTINGS_JSON) as json_file:
                    shooting_data = json.load(json_file)

                keyed_shooting_data = {}
                for shooting in shooting_data:
                    keyed_shooting_data[shooting["id"]] = shooting
                    shooting["date"] = parse(shooting["date"])

                print(f"Loading existing shooting data with {len(shooting_data)} incidents")
                current_url = driver.current_url
                if parse_page(current_url)=='empty':
                    continue
                else:
                    parse_page(current_url)
                    print("Sorting results")
                    updated_shooting_data = sorted(
                        keyed_shooting_data.values(), key=lambda row: row['date']
                    )
    
                    updated_shootings_compact = []
                    for shooting in updated_shooting_data:
                        shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
                        updated_shootings_compact.append(
                            [
                                shooting["id"],
                                shooting["date"],
                                shooting["killed"],
                                shooting["injured"],
    
                            ]
                        )
    
    
                    print("Writing results")
                    with open(SHOOTINGS_JSON, "w") as outfile:
                        json.dump(updated_shooting_data, outfile, indent=2)
    
                    with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
                        json.dump(updated_shootings_compact, outfile, separators=(",", ":"))
    
                    print(f"Done! {len(updated_shooting_data)} shootings found")
                    driver.close()
            else:
                break
            


Loading existing shooting data with 1175 incidents
https://www.gunviolencearchive.org/query/ec1923c8-8a2c-4632-86d5-b2781bfba5cd
New shooting found: 1204113, 2018-09-01 00:00:00
New shooting found: 1202105, 2018-09-01 00:00:00
New shooting found: 1201973, 2018-09-01 00:00:00
New shooting found: 1201640, 2018-09-01 00:00:00
https://www.gunviolencearchive.org/query/ec1923c8-8a2c-4632-86d5-b2781bfba5cd
Sorting results
Writing results
Done! 1179 shootings found
Loading existing shooting data with 1179 incidents
https://www.gunviolencearchive.org/query/408a1658-6f1b-422b-a295-5fb66e0ebd7d
New shooting found: 1201905, 2018-09-02 00:00:00
New shooting found: 1563922, 2018-09-02 00:00:00
New shooting found: 1204130, 2018-09-02 00:00:00
New shooting found: 1202474, 2018-09-02 00:00:00
https://www.gunviolencearchive.org/query/408a1658-6f1b-422b-a295-5fb66e0ebd7d
Sorting results
Writing results
Done! 1183 shootings found
Loading existing shooting data with 1183 incidents
https://www.gunviolencear

Sorting results
Writing results
Done! 1241 shootings found
Loading existing shooting data with 1241 incidents
https://www.gunviolencearchive.org/query/b720984a-3a13-465c-ae71-fdb4fb5fc8f6
New shooting found: 1215198, 2018-09-20 00:00:00
New shooting found: 1214045, 2018-09-20 00:00:00
https://www.gunviolencearchive.org/query/b720984a-3a13-465c-ae71-fdb4fb5fc8f6
Sorting results
Writing results
Done! 1243 shootings found
Loading existing shooting data with 1243 incidents
https://www.gunviolencearchive.org/query/ac3e8d0d-a3cc-4ddb-8097-25f8c9177585
empty
https://www.gunviolencearchive.org/query/ac3e8d0d-a3cc-4ddb-8097-25f8c9177585
empty
Sorting results
Writing results
Done! 1243 shootings found
Loading existing shooting data with 1243 incidents
https://www.gunviolencearchive.org/query/fd6e56ed-3fcf-43c1-882d-05b9d5661439
New shooting found: 1215761, 2018-09-22 00:00:00
New shooting found: 1216831, 2018-09-22 00:00:00
New shooting found: 1216411, 2018-09-22 00:00:00
New shooting found: 121

https://www.gunviolencearchive.org/query/ed2277e5-7217-4d10-8a88-3bff9ffc4a4c
Sorting results
Writing results
Done! 1310 shootings found
Loading existing shooting data with 1310 incidents
https://www.gunviolencearchive.org/query/87b19f8d-7242-449c-8913-c58a7f21acfc
New shooting found: 1230055, 2018-10-09 00:00:00
New shooting found: 1229241, 2018-10-09 00:00:00
New shooting found: 1229019, 2018-10-09 00:00:00
New shooting found: 1230175, 2018-10-09 00:00:00
https://www.gunviolencearchive.org/query/87b19f8d-7242-449c-8913-c58a7f21acfc
Sorting results
Writing results
Done! 1314 shootings found
Loading existing shooting data with 1314 incidents
https://www.gunviolencearchive.org/query/b6ec2c3f-d730-4ff4-981f-a06935896ee0
New shooting found: 1230462, 2018-10-10 00:00:00
New shooting found: 1229736, 2018-10-10 00:00:00
New shooting found: 1229879, 2018-10-10 00:00:00
https://www.gunviolencearchive.org/query/b6ec2c3f-d730-4ff4-981f-a06935896ee0
Sorting results
Writing results
Done! 1317 shoo

https://www.gunviolencearchive.org/query/745f4b7e-e493-474e-bc5f-81813509441e
Sorting results
Writing results
Done! 1380 shootings found
Loading existing shooting data with 1380 incidents
https://www.gunviolencearchive.org/query/1ae8b4b5-4ef5-4bec-9e6f-41b1d2deaaea
New shooting found: 1242733, 2018-10-27 00:00:00
New shooting found: 1242201, 2018-10-27 00:00:00
New shooting found: 1243099, 2018-10-27 00:00:00
https://www.gunviolencearchive.org/query/1ae8b4b5-4ef5-4bec-9e6f-41b1d2deaaea
Sorting results
Writing results
Done! 1383 shootings found
Loading existing shooting data with 1383 incidents
https://www.gunviolencearchive.org/query/929580fb-5f1a-4dac-bb45-59225155fe96
New shooting found: 1242882, 2018-10-28 00:00:00
New shooting found: 1242783, 2018-10-28 00:00:00
New shooting found: 1242779, 2018-10-28 00:00:00
https://www.gunviolencearchive.org/query/929580fb-5f1a-4dac-bb45-59225155fe96
Sorting results
Writing results
Done! 1386 shootings found
Loading existing shooting data with 1

https://www.gunviolencearchive.org/query/990ae498-c8da-4790-ab60-4febf84678c7
Sorting results
Writing results
Done! 1451 shootings found
Loading existing shooting data with 1451 incidents
https://www.gunviolencearchive.org/query/91f6854f-a754-4827-8f73-b77a064f4016
New shooting found: 1255350, 2018-11-14 00:00:00
New shooting found: 1255672, 2018-11-14 00:00:00
New shooting found: 1255632, 2018-11-14 00:00:00
New shooting found: 1257251, 2018-11-14 00:00:00
New shooting found: 1256230, 2018-11-14 00:00:00
https://www.gunviolencearchive.org/query/91f6854f-a754-4827-8f73-b77a064f4016
Sorting results
Writing results
Done! 1456 shootings found
Loading existing shooting data with 1456 incidents
https://www.gunviolencearchive.org/query/c8e334f5-8674-4408-b8bd-6b964b28875a
New shooting found: 1256595, 2018-11-15 00:00:00
New shooting found: 1256271, 2018-11-15 00:00:00
New shooting found: 1256752, 2018-11-15 00:00:00
New shooting found: 1256731, 2018-11-15 00:00:00
New shooting found: 1260118

New shooting found: 1267109, 2018-11-29 00:00:00
New shooting found: 1266173, 2018-11-29 00:00:00
New shooting found: 1265612, 2018-11-29 00:00:00
New shooting found: 1266388, 2018-11-29 00:00:00
https://www.gunviolencearchive.org/query/a93e9987-4ceb-415e-a16d-64a99e70ad64
Sorting results
Writing results
Done! 1536 shootings found
Loading existing shooting data with 1536 incidents
https://www.gunviolencearchive.org/query/0c87a6a8-291a-48d2-a15c-4f1cb2237df0
New shooting found: 1266675, 2018-11-30 00:00:00
New shooting found: 1266905, 2018-11-30 00:00:00
New shooting found: 1316796, 2018-11-30 00:00:00
New shooting found: 1266841, 2018-11-30 00:00:00
New shooting found: 1269868, 2018-11-30 00:00:00
New shooting found: 1267534, 2018-11-30 00:00:00
New shooting found: 1268594, 2018-11-30 00:00:00
New shooting found: 1513163, 2018-11-30 00:00:00
https://www.gunviolencearchive.org/query/0c87a6a8-291a-48d2-a15c-4f1cb2237df0
Sorting results
Writing results
Done! 1544 shootings found
Loading e

New shooting found: 1278908, 2018-12-15 00:00:00
New shooting found: 1278045, 2018-12-15 00:00:00
New shooting found: 1279061, 2018-12-15 00:00:00
New shooting found: 1280781, 2018-12-15 00:00:00
New shooting found: 1278409, 2018-12-15 00:00:00
https://www.gunviolencearchive.org/query/91d79932-177a-422b-89b1-ee5fabc33fe7
Sorting results
Writing results
Done! 1618 shootings found
Loading existing shooting data with 1618 incidents
https://www.gunviolencearchive.org/query/fac9f77f-e659-4a89-b01c-0ac83c67cda5
New shooting found: 1278429, 2018-12-16 00:00:00
New shooting found: 1279112, 2018-12-16 00:00:00
New shooting found: 1283283, 2018-12-16 00:00:00
https://www.gunviolencearchive.org/query/fac9f77f-e659-4a89-b01c-0ac83c67cda5
Sorting results
Writing results
Done! 1621 shootings found
Loading existing shooting data with 1621 incidents
https://www.gunviolencearchive.org/query/19d20cb2-39e0-436e-b0f7-e1bebe1b59c4
New shooting found: 1280144, 2018-12-17 00:00:00
https://www.gunviolencearch

## Test code 

In [29]:
# # TEST LOOP:

# year = ["2019","2020"]
# month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# count=0
# for years in range(len(year)): 
    
#     for months in range(len(month)):
        
                
#         print(day)
        
           
            



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2

In [ ]:
# Opening and reading files test code:
# start_year = 2014
# current_year = datetime.now().year
# with open(SHOOTINGS_JSON) as json_file:
#     shooting_data = json.load(json_file)

# keyed_shooting_data = {}
# for shooting in shooting_data:
#     keyed_shooting_data[shooting["id"]] = shooting
#     shooting["date"] = parse(shooting["date"])

# print(f"Loading existing shooting data with {len(shooting_data)} incidents")

In [15]:
# start_year = 2014
# current_year = datetime.now().year
# with open(SHOOTINGS_JSON) as json_file:
#     shooting_data = json.load(json_file)

# keyed_shooting_data = {}
# for shooting in shooting_data:
#     keyed_shooting_data[shooting["id"]] = shooting
#     shooting["date"] = parse(shooting["date"])

# print(f"Loading existing shooting data with {len(shooting_data)} incidents")

# # Insert any GVA date query url link into the parse page function to test the code
# parse_page('https://www.gunviolencearchive.org/query/b6bf79c7-0651-4713-8eca-cae8e66e150f')
# print("Sorting results")
# updated_shooting_data = sorted(
#     keyed_shooting_data.values(), key=lambda row: row['date']
# )

# updated_shootings_compact = []
# for shooting in updated_shooting_data:
#     shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
#     updated_shootings_compact.append(
#         [
#             shooting["id"],
#             shooting["date"],
#             shooting["killed"],
#             shooting["injured"],

#         ]
#     )


# print("Writing results")
# with open(SHOOTINGS_JSON, "w") as outfile:
#     json.dump(updated_shooting_data, outfile, indent=2)

# with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
#     json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

# print(f"Done! {len(updated_shooting_data)} shootings found")

Loading existing shooting data with 56112 incidents
https://www.gunviolencearchive.org/query/b6bf79c7-0651-4713-8eca-cae8e66e150f
Sorting results
Writing results
Done! 56112 shootings found


In [38]:
# y = ['2020']
# m = ['Apr']
# d = [10]
# for ys in range(len(y)):
#     for ms in range(len(m)):
#         for ds in range(len(d)):
#             c_date = '{}-{}-{}'.format(y[ys],m[ms],d[ds])
#             c_date = datetime.strptime(c_date,"%Y-%b-%d")
#             day_difference = (datetime.now() - c_date).days
#             print(day_difference)
#             if (day_difference >3):
#                 print(c_date)
#             else:
#                 print("cats")

#### Geocoder test code

In [ ]:
# for year in range(start_year, current_year + 1):
#     print(f"scraping year {year}")
#     page = 0
#     done_importing = False
#     while not done_importing:
#         results = scrape_results(year, page, headers)
#         new_results_count = merge_shooting_data(results)
#         done_importing = new_results_count == 0
#         page += 1
#         sleep(1)
#     print(f"Done importing {year}")

# # print("Geocoding missing data")
# # for shooting in keyed_mass_shooting_data.values():
# #     if not shooting.get("lat") or not shooting.get("lng"):
# #         loc = geocode(shooting["address"], shooting["city"], shooting["state"])
# #         shooting["lat"] = loc.latitude
# #         shooting["lng"] = loc.longitude
# #         print(f"geocoded {loc.address}: {loc.latitude}, {loc.longitude}")

#### Update results method test (did not use in implementation)

In [ ]:
# def update_results():

#     start_year = 2014
#     current_year = datetime.now().year
#     with open(ACCIDENTAL_SHOOTINGS_JSON) as json_file:
#         accidental_shooting_data = json.load(json_file)

#     keyed_accidental_shooting_data = {}
#     for shooting in accidental_shooting_data:
#         keyed_accidental_shooting_data[shooting["id"]] = shooting
#         shooting["date"] = parse(shooting["date"])

#     print(f"Loading existing accidental shooting data with {len(accidental_shooting_data)} incidents")
#     print("Sorting results")
#     updated_shooting_data = sorted(
#         keyed_accidental_shooting_data.values(), key=lambda row: row['date']
#     )

#     updated_shootings_compact = []
#     for shooting in updated_shooting_data:
#         shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
#         updated_shootings_compact.append(
#             [
#                 shooting["id"],
#                 shooting["date"],
#                 shooting["killed"],
#                 shooting["injured"],

#             ]
#         )


#     print("Writing results")
#     with open(ACCIDENTAL_SHOOTINGS_JSON, "w") as outfile:
#         json.dump(updated_shooting_data, outfile, indent=2)

#     with open(ACCIDENTAL_SHOOTINGS_COMPACT_JSON, "w") as outfile:
#         json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

#     print(f"Done! {len(updated_shooting_data)} shootings found")